In [1]:
import numpy as np
from jetto_mobo import ecrh, genetic_algorithm
import plotly.graph_objects as go
import h5py

n_steps = 10
batch_size = 5
n_objectives = 7
n_parameters = 10
n_radial_points = 150

parameters = np.empty((n_steps * batch_size, n_parameters))
objective_values = np.empty((n_steps * batch_size, n_objectives))
converged_ecrh = np.empty((n_steps * batch_size, 150))
converged_q = np.empty((n_steps * batch_size, 150))

with h5py.File("../data/mobo/sum_of_gaussians/bayesopt.hdf5") as hdf5:
    for i in np.arange(n_steps):
        for j in np.arange(batch_size):
            objective_values[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/value"][j]
            parameters[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/ecrh_parameters"][j]
            converged_ecrh[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/converged_ecrh"][j]
            converged_q[i*batch_size + j] = hdf5[f"bayesopt/{i+1}/converged_q"][j]


19:00:48 [WARNING] Python module 'omas' not found. Submodule 'jams' needs it @jams.py:14


In [2]:
from jetto_mobo.utils import get_pareto_dominant_mask

pareto_dominant_mask = get_pareto_dominant_mask(objective_values)
n_pareto_optimal_points = np.sum(pareto_dominant_mask)

In [10]:
from typing import Iterable
import numpy as np
from plotly import graph_objects as go 
from plotly.subplots import make_subplots
import plotly

figure = make_subplots(rows=2, cols=2, specs=[[{'rowspan': 2, 'type': 'polar'}, {}], [{}, {}]])

n_pareto_optimal_points = np.sum(pareto_dominant_mask)
i = 0
for objective_value, ecrh, q in zip(objective_values[pareto_dominant_mask],
                                    converged_ecrh[pareto_dominant_mask],
                                    converged_q[pareto_dominant_mask]):
    figure.add_traces(
        [
            go.Scatterpolar(
                r=objective_value,
                theta=["|q(0) - min(q)|",
                        "|min(q) - 2.2|",
                        "|argmin(q)|",
                        "Non-monotonic<br>fraction of q",
                        "Non-monotonic<br>fraction of dq",
                        "rho of q=3",
                        "rho of q=4"],
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=False,
                fill="toself",
                line_color=plotly.colors.qualitative.Alphabet[i]
            ),
            go.Scatter(
                x=np.linspace(0, 1, n_radial_points),
                y=ecrh,
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=False,
                mode='lines',
                line_color=plotly.colors.qualitative.Alphabet[i],
            ),
            go.Scatter(
                x=np.linspace(0, 1, n_radial_points),
                y=q,
                legendgroup=i,
                name=f"Solution {i}",
                showlegend=True,
                mode='lines',
                line_color=plotly.colors.qualitative.Alphabet[i],
            )
        ],
        rows=[1, 1, 2],
        cols=[1, 2, 2]
    )
    i+=1
    
figure.show()